# Modelo de recomendación de juegos

En esta notebook se experimenta para crear dos modelos de recomendación de juegos. 

1. El primer modelo emplea relaciones entre juegos para recomendar títulos similares basados en un juego ingresado. 
2. El segundo modelo utiliza un enfoque de filtro usuario-juego, recomendando juegos basados en usuarios similares a uno dado. 

Ambos modelos utilizan algoritmos de memoria para el "filtrado colaborativo", buscando usuarios similares y empleando sus preferencias para predecir las valoraciones del usuario activo.

## Importaciones

In [79]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

## Datos a utilizar

Se leen los datos que se prepararon luego del EDA y se convierten en dataframe para ser utilizados por el modelo.

In [80]:
df_modelo = pd.read_parquet('../Datasets/recomendacion.parquet')
df_modelo.head(3)

,user_id,item_id,rating,item_name
0,76561197970982479,1250,3,Killing Floor
1,js41637,251610,3,Barbie™ Dreamhouse Party™
2,doctr,250320,5,The Wolf Among Us


## Función de recomendación según un juego (item-item)

Se propone una función para recomendar 5 juegos basados en la similitud con un juego dado. La función toma el nombre de un juego, ordena la matriz de similitud de juegos y elige los 5 más similares, excluyendo el juego de entrada. Luego, imprime esta lista de juegos similares al especificado.

#### Preprocesar los datos y almacenar la información relevante en un nuevo DataFrame.

In [81]:
# Obtenemos una tabla con registros unicos de cada juego
unique_item_ids = df_modelo.drop_duplicates(subset='item_id')
unique_item_ids.drop(columns=['user_id','rating'],inplace=True)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_8412\1305628876.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_item_ids.drop(columns=['user_id','rating'],inplace=True)


In [82]:
# reducimos el data por una cuestion de espacio en render, pero sin que afecte la calidad de los datos
df_modelo = df_modelo.sample(frac=0.125, random_state=1)

In [76]:
# Calcular la matriz de similitud de cosenos y preprocesar los datos
item_vectors = df_modelo.pivot_table(index='item_id', columns='user_id', values='rating', aggfunc='mean').fillna(0)

## Función de recomendación según un usuario (user-item)

La siguiente función tiene la finalidad de Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.

In [77]:
# Preprocesamiento para el cálculo de similitud de usuarios y mapeo de item_id con item_name
user_vectors = df_modelo.pivot(index='user_id', columns='item_id', values='rating').fillna(0)

## Carga de las matrices para recomendar en la API

Para poder utilizar las funciones **recomendacion_juego** y **recomendacion_usuario** es necesario consumir las matrices **item_vectors** y **user_vectors**. Por ello, se guardan, en este caso, en formato parquet los que permite una compresión y codificación eficiente.

In [78]:
item_vectors.to_parquet('../Datasets/item_vectors.parquet')
unique_item_ids.to_parquet('../Datasets/unique_item_ids.parquet')
user_vectors.to_parquet('../Datasets/user_vectors.parquet')

print('Se guardaron correctamente')

Se guardaron correctamente
